ref : https://colab.research.google.com/drive/1wu3i6UpLk7HjaPJidkMdAY7Ob84mQdNj?usp=sharing

https://huggingface.co/docs/peft/main/en/task_guides/seq2seq-prefix-tuning


In [3]:
!pip install -q peft transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = "cuda"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
label_column = "text_label"

max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

sentences_allagree 데이터셋 : 금융 뉴스 감정 (sentiment) 분류 데이터셋

In [5]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size = 0.1)
dataset["validation"] = dataset["test"]

del dataset["test"]

print(dataset["train"][0])

classes = dataset["train"].features["label"].names
dataset = dataset.map(lambda x: {label_column: [classes[label] for label in x["label"]]}, batched=True, num_proc=1)

print(dataset["train"][0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

{'sentence': 'Nordic banks have already had to write off sizable loans in Latvia , with Swedbank , Nordea , DnB NOR and SEB reporting combined losses in excess of $ 1.35 billion in the period 2007 to 2010 against a backdrop of near economic meltdown in Latvia .', 'label': 0}


Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Nordic banks have already had to write off sizable loans in Latvia , with Swedbank , Nordea , DnB NOR and SEB reporting combined losses in excess of $ 1.35 billion in the period 2007 to 2010 against a backdrop of near economic meltdown in Latvia .', 'label': 0, 'text_label': 'negative'}


데이터셋 tokenization

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples) :
  inputs = examples[text_column]
  targets = examples[label_column]

  model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")

  labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
  labels = labels["input_ids"]
  labels[labels == tokenizer.pad_token_id] = -100
  model_inputs["labels"] = labels

  return model_inputs

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running Tokenizer on Dataset",
)

Running Tokenizer on Dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running Tokenizer on Dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [8]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

peft 모델 설정

In [9]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = (len(train_dataloader) * num_epochs)
)

학습 진행

In [14]:
model = model.to(device)

for epoch in range(num_epochs) :
  model.train()
  total_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader)) :
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    total_loss += loss.detach().float()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

  model.eval()
  eval_loss = 0
  eval_preds = []
  for step, batch in enumerate(tqdm(eval_dataloader)) :
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad() :
      outputs = model(**batch)
    loss = outputs.loss
    eval_loss += loss.detach().float()
    eval_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
    )

  eval_epoch_loss = eval_loss / len(eval_dataloader)
  eval_ppl = torch.exp(eval_epoch_loss)
  train_epoch_loss = total_loss / len(train_dataloader)
  train_ppl = torch.exp(train_epoch_loss)

  print(f"{epoch = }: {train_ppl = } {train_epoch_loss = } {eval_ppl = } {eval_epoch_loss = }")

100%|██████████| 29/29 [00:07<00:00,  4.14it/s]


epoch = 0: train_ppl = tensor(6.5666, device='cuda:0') train_epoch_loss = tensor(1.8820, device='cuda:0') eval_ppl = tensor(1.1506, device='cuda:0') eval_epoch_loss = tensor(0.1403, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  4.12it/s]


epoch = 1: train_ppl = tensor(1.1350, device='cuda:0') train_epoch_loss = tensor(0.1266, device='cuda:0') eval_ppl = tensor(1.0762, device='cuda:0') eval_epoch_loss = tensor(0.0734, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.26it/s]


epoch = 2: train_ppl = tensor(1.0823, device='cuda:0') train_epoch_loss = tensor(0.0791, device='cuda:0') eval_ppl = tensor(1.0600, device='cuda:0') eval_epoch_loss = tensor(0.0583, device='cuda:0')


100%|██████████| 29/29 [00:06<00:00,  4.27it/s]


epoch = 3: train_ppl = tensor(1.0780, device='cuda:0') train_epoch_loss = tensor(0.0751, device='cuda:0') eval_ppl = tensor(1.0584, device='cuda:0') eval_epoch_loss = tensor(0.0567, device='cuda:0')


100%|██████████| 29/29 [00:07<00:00,  4.00it/s]

epoch = 4: train_ppl = tensor(1.0578, device='cuda:0') train_epoch_loss = tensor(0.0562, device='cuda:0') eval_ppl = tensor(1.0478, device='cuda:0') eval_epoch_loss = tensor(0.0466, device='cuda:0')


모델 평가

In [16]:
correct = 0
total = 0

for pred, true in zip(eval_preds, dataset["validation"]["text_label"]) :
  if pred.strip() == true.strip() :
    correct += 1
  total += 1

accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=97.3568281938326 % on the evaluation dataset
eval_preds[:10]=['negative', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'positive', 'negative', 'negative', 'positive']
dataset['validation']['text_label'][:10]=['negative', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'positive', 'negative', 'negative', 'positive']


모델 테스트

In [23]:
inputs = tokenizer(
    "America goverment will decrease interest rate",
    return_tensors="pt"
)

In [24]:
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
